In [1]:
import os

try:
    has_changed_dir
except:
    has_changed_dir = False

try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

if IN_COLAB:
    %pip install datasets
    %pip install translate-toolkit
    #%pip install bitsandbytes

    !git clone https://github.com/MartinKirkegaardDK/KDS_MI.git

    if not has_changed_dir:
        os.chdir('KDS_MI/refactor')
        has_changed_dir = True
else:
    if not has_changed_dir:
        os.chdir('.')
        has_changed_dir = True

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from make_steering_vectors import main

In [7]:
main(
    model_url='EleutherAI/pythia-14m',
    target_language='da',
    complement_languages=['en', 'sv', 'is', 'nb'],
    device='cpu',
    out_folder='steering_vectors'
)

 32%|███▏      | 6/19 [00:12<00:26,  2.03s/it]


In [8]:
from utils.hooking import HookManager
from utils.compatibility import *
from utils.data import load_steering_vector

from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_url='EleutherAI/pythia-14m'
device = 'cpu'

model = AutoModelForCausalLM.from_pretrained(model_url).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_url)

In [29]:
input_ = 'The bacterium that was isolated from Mohammed\'s bloodstream was'

address = HookAddress.attention_post.layer(3)

steering_vector = load_steering_vector('da', address, model)

with HookManager(model) as hook_manager:
    hook_manager.steer(
        hook_address=address,
        steering_vector=steering_vector,
        scalar=10
    )

    tokenized = tokenizer(input_, return_tensors='pt').to(Device.device(model))
    undecoded_output = model.generate(
                inputs=tokenized.input_ids, 
                max_length=100, 
                temperature=0.7, 
                top_p=0.9, 
                do_sample=True,

            )

print(tokenizer.decode(undecoded_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


The bacterium that was isolated from Mohammed's bloodstream was connected with the production of a special soap, which was also not performed in the same manner as the other other other other other other other other other other other other other other other other other actors in the series.

The host, which was not at all a very strong, and the test, and the testing was performed in the series, and was tested for the testing.

The test set for an initial test, with the test


In [12]:
steering_vector